In [11]:
import numpy as np
from PIL import Image
import os
import pandas as pd
from torchvision import transforms
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
import os

In [6]:
def resize_image(image, width, height):
    return image.resize((width, height))

In [7]:
def crop_image(image, x, y, width, height):
  return image.crop((x, y, x + width,  x + height))

In [49]:
def padronizar_imagem(imagem,):

    # Carregar a imagem
    imagem = Image.open(imagem)
    
    # Converter a imagem redimensionada para um array numpy
    imagem_array = np.array(imagem).astype(np.float32)
    
    # Calcular a média e o desvio padrão dos valores dos pixels
    mean = np.mean(imagem_array)
    std = np.std(imagem_array)
    
    # Normalizar a imagem usando a fórmula (x = (x - mean) / std)
    imagem_padronizada = (imagem_array - mean) / std
    
    return imagem_padronizada

In [12]:
IMG_MEAN = .5862785803043838
IMG_STD = .27950088968644304
# IMG_MEAN = .44666268853478000
# IMG_STD = .3144343342216960

In [ ]:
def padronizar_imagem(image):
    image_array = np.array(image)
    normalized_image_array = (image_array - IMG_MEAN) / IMG_STD

    # Converter o array NumPy normalizado de volta para imagem PIL (e converter para uint8)
    normalized_image = Image.fromarray(normalized_image_array.astype(np.uint8))
    return normalized_image

In [14]:
def normalize(image):
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(mean=[IMG_MEAN], std=[IMG_STD]),
        transforms.ToPILImage()
    ])

    # Aplicar a transformação
    transformed_image = transform(image)
    #trans_image = trans(image)

    # Converter tensor para numpy array
    #to_pil = transforms.ToPILImage()
    #image_pil = to_pil(transformed_image)
    
    return transformed_image

In [15]:
def calculate_mean_and_std_normal(image):

    img_array = np.array(image)

    non_black_pixels = img_array / 255.

    mean = np.mean(non_black_pixels)
    std = np.std(non_black_pixels)
    max_value = np.max(non_black_pixels)
    min_value = np.min(non_black_pixels)
        

    results = [
                mean,
                std,
                max_value,
                min_value
    ]

    return results

In [16]:
def calculate_mean_and_std(folder):
    results = []
    corrupt_files = []

    for root, dirs, files in os.walk(folder):
        for file in files:
            if file.lower().endswith('.jpg'):
                try:
                    img_path = os.path.join(root, file)
                    img = Image.open(img_path).convert('L')
                except Exception as e:
                    print(f"Erro ao abrir {file}: {e}")
                    corrupt_files.append(os.path.abspath(file))
                    continue

                # img = resize_image(img, 390, 320)

                # x, y, width, height = crop_coords
                # cropped_img = crop_image(img, x, y, width, height)

                img_array = np.array(img)

                pixels = img_array / 255.

                mean = np.mean(pixels)
                std = np.std(pixels)
                max_value = np.max(pixels)
                min_value = np.min(pixels)

                if np.isnan(mean) or np.isnan(std):
                    print(f"Encontrado NaN em {file} com mean: {mean}, std: {std}")
                    continue
                
                mean_normal, std_normal, max_value_normal, min_value_normal = calculate_mean_and_std_normal(normalize(img))

                results.append({
                    "file_path": file,
                    "mean": mean,
                    "std": std,
                    "max_value": max_value,
                    "min_value": min_value,
                    "mean_normal": mean_normal,
                    "std_normal": std_normal,
                    "max_value_normal": max_value_normal,
                    "min_value_normal": min_value_normal
                })
                
    df = pd.DataFrame(results)

    return  df

In [17]:
path = r'../mimic-cxr/mimic-cxr-images/p10/p10000032'

df = calculate_mean_and_std(path)

print(df)
df.to_csv('resultados.csv', index=False)

                                          file_path      mean       std  \
0  02aa804e-bde0afdd-112c0b34-7bc16630-4e384014.jpg  0.372197  0.332217   
1  174413ec-4ec4c1f7-34ea26b7-c5f994f8-79ef1962.jpg  0.362351  0.332439   
2  2a2277a9-b0ded155-c0de8eb9-c124d10e-82c5caab.jpg  0.492572  0.295344   
3  e084de3b-be89b11e-20fe3f9f-9c8d8dfe-4cfd202c.jpg  0.491098  0.290418   
4  68b5c4b1-227d0485-9cc38c3f-7b84ab51-4b472714.jpg  0.484492  0.297349   
5  fffabebf-74fd3a1f-673b6b41-96ec0ac9-2ab69818.jpg  0.443380  0.314677   
6  ea030e7a-2e3b1346-bc518786-7a8fd698-f673b44c.jpg  0.446663  0.314434   

   max_value  min_value  mean_normal  std_normal  max_value_normal  \
0        1.0        0.0     0.590483    0.319025               1.0   
1        1.0        0.0     0.598006    0.317785               1.0   
2        1.0        0.0     0.490205    0.300426               1.0   
3        1.0        0.0     0.508371    0.290389               1.0   
4        1.0        0.0     0.492962    0.296114 